In [1]:
### 1. 크롤링/전처리를 위한 필수 패키지 import ###

import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
from bs4 import BeautifulSoup
import requests

url = "https://m.stock.naver.com/marketindex/home/exchangeRate/exchange"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

country_list = []
rate_list = []
updown_list = []
updown_percentage_list = []

# 주요 국가 추출
country_data = soup.findAll(class_="MainListItem_name__2Nl6J")
rate_data = soup.findAll(class_="MainListItem_price__dP8R6")
fluctuation_data = soup.findAll(class_="Fluctuation_fluctuation__9UU9_")

# Extracting country names
for country in country_data:
    country_list.append(country.text)

# Extracting rates
for rate in rate_data:
    rate_list.append(rate.text)

# Extracting fluctuations and percentages
for i in range(len(fluctuation_data)):
    if i % 2 == 0:
        updown_list.append(fluctuation_data[i].text)
    else:
        updown_percentage_list.append(fluctuation_data[i].text)

# print("Country list:", country_list)
# print("Rate list:", rate_list)
# print("Up/Down list:", updown_list)
# print("Up/Down Percentage list:", updown_percentage_list)

df = pd.DataFrame({'country': country_list, 'rate': rate_list, 'updown': updown_list, 'percent': updown_percentage_list})
print(df)

     country      rate  updown  percent
0     미국 USD  1,284.50    5.50   +0.43%
1     유럽 EUR  1,412.05    2.46   +0.17%
2     일본 JPY    900.42    3.47   +0.39%
3     중국 CNY    179.14    0.31   +0.17%
4      유로/달러    1.0995  0.0020   -0.18%
5  영국 파운드/달러    1.2836  0.0009   -0.07%


In [3]:
import pandas as pd
import MySQLdb
from bs4 import BeautifulSoup
import requests

url = "https://m.stock.naver.com/marketindex/home/exchangeRate/exchange"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

country_list = []
rate_list = []
updown_list = []
updown_percentage_list = []

# 주요 국가 추출
country_data = soup.findAll(class_="MainListItem_name__2Nl6J")
rate_data = soup.findAll(class_="MainListItem_price__dP8R6")
fluctuation_data = soup.findAll(class_="Fluctuation_fluctuation__9UU9_")

# Extracting country names
for country in country_data:
    country_list.append(country.text)

# Extracting rates
for rate in rate_data:
    rate_list.append(rate.text)

# Extracting fluctuations and percentages
for i in range(len(fluctuation_data)):
    if i % 2 == 0:
        updown_list.append(fluctuation_data[i].text)
    else:
        updown_percentage_list.append(fluctuation_data[i].text)

# Create DataFrame
df = pd.DataFrame({'country': country_list, 'rate': rate_list, 'updown': updown_list, 'percent': updown_percentage_list})

# Database connection
conn = MySQLdb.connect(
    user="scott",
    password="tiger",
    host="localhost",
    db="project"
)

# Create table if it doesn't exist
cursor = conn.cursor()
cursor.execute("""
    CREATE TABLE IF NOT EXISTS main_rate (
        id INT AUTO_INCREMENT PRIMARY KEY,
        country VARCHAR(255),
        rate VARCHAR(255),
        updown VARCHAR(255),
        percent VARCHAR(255)
    )
""")

# DataFrame의 데이터를 MySQL 테이블에 삽입
for index, row in df.iterrows():
    cursor.execute(
        "INSERT INTO main_rate (`country`, `rate`, `updown`, `percent`) VALUES (%s, %s, %s, %s)",
        (row['country'], row['rate'], row['updown'], row['percent'])
    )

conn.commit()
conn.close()
